In [8]:
import numpy as np
import pandas as pd

#Grandezza campione
sample_size = 1000

#razze e tipi di mangime 
razze = ["Leghorn", "Rhode Island Red", "Sussex", "Plymouth Rock"]
mangimi = ["Type A", "Type B", "Type C"]

# Creazinoe dataset
np.random.seed(42)  # For reproducibility

# Genera dati casuali per le feature
razza = np.random.choice(razze, size=sample_size)
mangime = np.random.choice(mangimi, size=sample_size)
temperatura = np.random.uniform(20, 35, size=sample_size)  # temperatura in Celsius
eta = np.random.randint(5, 75, size=sample_size)  # età in mesi

# Initialize array con peso e uova a zero
peso = np.zeros(sample_size)
q_uova_mensili = np.zeros(sample_size)

#applicazione dei pattern
for i in range(sample_size):
    breed = razza[i]
    feed = mangime[i]
    age = eta[i]
    
    # il peso bassato sulla razza della gallina
    if breed == "Leghorn":
        peso[i] = np.random.uniform(1500, 2500)  #Peso leggero
    elif breed == "Rhode Island Red":
        peso[i] = np.random.uniform(2500, 3500)  #Peso massimo
    elif breed == "Sussex":
        peso[i] = np.random.uniform(2000, 3000)  #Peso medio
    else:  # Plymouth Rock
        peso[i] = np.random.uniform(2200, 3200) #Peso bantam
    
    # Adjust weight based on feed type
    if feed == "Type A":
        peso[i] += np.random.uniform(100, 300)  # Type A ha galline più grosse
    elif feed == "Type B":
        peso[i] += np.random.uniform(200, 400)  # Type B incrementa il peso
    
    # pattern produzzione uova in base all'età
    if age <= 12:
        q_uova_mensili[i] = np.random.poisson(lam=8)  
    elif age <= 24:
        q_uova_mensili[i] = np.random.poisson(lam=6)  
    else:
        q_uova_mensili[i] = np.random.poisson(lam=4)  
    
    # aggiustare produzione in base alla razza
    if breed == "Leghorn":
        q_uova_mensili[i] += np.random.randint(2, 4)  # Livorno depongono più uova
    elif breed == "Rhode Island Red":
        q_uova_mensili[i] -= np.random.randint(0, 2)  # Rhode island red di meno
        if q_uova_mensili[i] < 0:
            q_uova_mensili[i] = 0 #non puo essere minore di zero
    
    if feed == "Type A":
        q_uova_mensili[i] += np.random.randint(0, 2)  # Type A producono più uova
    elif feed == "Type B":
        q_uova_mensili[i] -= np.random.randint(0, 1)  # Type B meno uova
        if q_uova_mensili[i] < 0:
            q_uova_mensili[i] = 0 #non puo essere minore di zero
    
    # effetto temperatura sula produzione
    if temperatura[i] < 22 or temperatura[i] > 30:
        q_uova_mensili[i] -= np.random.randint(0, 2)  # Extreme temperatures reduce egg production
        if q_uova_mensili[i] < 0:
            q_uova_mensili[i] = 0 #non puo essere minore di zero

# Creazione dataframe
data = {
    "eta": eta,
    "razza": razza,
    "mangime": mangime,
    "temperatura": temperatura,
    "peso": peso,
    "q_uova_mensili": q_uova_mensili
}

df = pd.DataFrame(data)

# Show the first few rows of the dataset
#print(df.head())

In [9]:
data = pd.read_csv("dataset.csv", index_col = 0)

print(data.head())  #reperimento dataset

   eta          razza mangime  temperatura         peso  q_uova_mensili
0   24         Sussex  Type B    33.417199  2713.508396             3.0
1   38  Plymouth Rock  Type C    34.872130  2313.303505             5.0
2   30        Leghorn  Type A    20.922755  2308.840657             7.0
3    9         Sussex  Type A    33.243594  3110.881379             4.0
4   65         Sussex  Type A    27.741185  2486.098303             7.0


In [3]:
mangime_dict = {"Type A": 1, "Type B": 2, "Type C": 3}
data["mangime"] = data["mangime"].map(mangime_dict)
print(data.head())  #conversione feature categorica in feature numerica

   Unnamed: 0  eta          razza  mangime  temperatura         peso  \
0           0   24         Sussex        2    33.417199  2713.508396   
1           1   38  Plymouth Rock        3    34.872130  2313.303505   
2           2   30        Leghorn        1    20.922755  2308.840657   
3           3    9         Sussex        1    33.243594  3110.881379   
4           4   65         Sussex        1    27.741185  2486.098303   

   q_uova_mensili  
0             3.0  
1             5.0  
2             7.0  
3             4.0  
4             7.0  


In [4]:
razza_dict = {"Leghorn": 1, "Rhode Island Red": 0.5, "Sussex": 0.7, "Plymouth Rock": 0}
data["razza"] = data["razza"].map(razza_dict)
print(data.head())  #conversione feature categorica in feature numerica


   Unnamed: 0  eta  razza  mangime  temperatura         peso  q_uova_mensili
0           0   24    0.7        2    33.417199  2713.508396             3.0
1           1   38    0.0        3    34.872130  2313.303505             5.0
2           2   30    1.0        1    20.922755  2308.840657             7.0
3           3    9    0.7        1    33.243594  3110.881379             4.0
4           4   65    0.7        1    27.741185  2486.098303             7.0


In [5]:
from sklearn.model_selection import train_test_split
#divisione dei dati in dati train e dati test con stratificazione
train_set, test_set = train_test_split(data, test_size=0.2, stratify=data["razza"], random_state=42)

In [6]:
#divisione variabili indipendente da variabile dipendente
x_train, y_train = train_set.drop(columns=['q_uova_mensili']), train_set['q_uova_mensili']
x_test, y_test = test_set.drop(columns=['q_uova_mensili']), test_set['q_uova_mensili']

In [7]:
#Calcolo della Varianza 
varianza = np.var(y_train)
print(f"la varianza è : {varianza}")

la varianza è : 7.4919


In [8]:
from sklearn.linear_model import LinearRegression
#creazione modello di regressione multipla
mul = LinearRegression()
#fitting del modello
mul.fit(x_train, y_train)

LinearRegression()

In [9]:
from sklearn.metrics import classification_report, accuracy_score

y_train_pred = mul.predict(x_train)
y_test_pred = mul.predict(x_test)

In [10]:
from sklearn.metrics import mean_squared_error

mse_test = mean_squared_error(y_test, y_test_pred)

print(f"Mean Squared Error: {mse_test}")

Mean Squared Error: 6.756398280914194


In [11]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(max_depth=3, random_state=42)
regressor.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=3, random_state=42)

In [12]:
from sklearn.metrics import mean_squared_error
pred_tree = regressor.predict(x_test)
mse_tree = mean_squared_error(y_test, pred_tree)

print(f"Mean Squared Error: {mse_tree}")

Mean Squared Error: 6.485474465392046


In [13]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr.fit(x_train, y_train)

SVR()

In [14]:
pred_svr = svr.predict(x_test)
mse_svr = mean_squared_error(y_test, pred_svr)

print(f"Mean Squared Error: {mse_svr}")

Mean Squared Error: 7.505769110287365


In [15]:
#I modelli non sono buoni facciamo cross-validation e poi grid search per ogni modello
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [16]:
#Facciamo la cross-validation 5-fold per la regressione multipla
cv_mul = cross_val_score(mul, x_train, y_train, cv=5, scoring='neg_mean_squared_error')

# risultati
print(f"Cross-validation MSE scores: {cv_mul}")
print(f"Mean cross-validation MSE: {cv_mul.mean()}")

Cross-validation MSE scores: [-6.39187108 -5.41969165 -5.06198598 -5.87028104 -5.12606422]
Mean cross-validation MSE: -5.573978792952433


In [17]:
#Con la Grid Search definiamo i parametri che possiamo modificare che per la regressione multipla non sono molti
param_grid = {
    'fit_intercept': [True, False],  # Whether to calculate the intercept
}

# grid search con la cross-validation
grid_mul = GridSearchCV(estimator=mul, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit della grid search
grid_mul.fit(x_train, y_train)

# Best parameters found by GridSearchCV
print(f"parametro migliore: {grid_mul.best_params_}")
print(f"Risultato migliore: {-grid_mul.best_score_}")

parametro migliore: {'fit_intercept': True}
Risultato migliore: 5.573978792952433


In [18]:
#Calcola il modello migliore con la grid search
best_model_mul = grid_mul.best_estimator_
pred_cross_mul = best_model_mul.predict(x_test)

#Calcola MSE sul test set
mse = mean_squared_error(y_test, pred_cross_mul)
print(f"Mean Squared Error on Test Set: {mse}")

Mean Squared Error on Test Set: 6.756398280914194


In [19]:
#Facciamo la cross-validation 5-fold per il decision tree regressor
cv_reg = cross_val_score(regressor, x_train, y_train, cv=5, scoring='neg_mean_squared_error')

# risultati
print(f"Cross-validation MSE scores: {-cv_reg}")
print(f"Mean cross-validation MSE: {-cv_reg.mean()}")

Cross-validation MSE scores: [5.5895557  5.19369337 4.9178463  5.1718705  4.54086158]
Mean cross-validation MSE: 5.082765492576821


In [20]:
#Con la Grid Search definiamo i parametri che possiamo modificare che per la regressione multipla non sono molti
param_grid = {
    'max_depth': [None, 5, 10, 20],  # Limits the depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum samples required to be at a leaf node
    'max_features': [None, 'sqrt', 'log2'],  # Number of features to consider for splitting
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],  # Quality of split
    'splitter': ['best', 'random'],  # Strategy used to split at each node
}

# grid search con la cross-validation
grid_reg = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit della grid search
grid_reg.fit(x_train, y_train)

# Best parameters found by GridSearchCV
print(f"parametro migliore: {grid_reg.best_params_}")
print(f"Risultato migliore: {-grid_reg.best_score_}")

parametro migliore: {'criterion': 'poisson', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'random'}
Risultato migliore: 5.231067419630557


In [21]:
#Calcola il modello migliore con la grid search
best_model_reg = grid_reg.best_estimator_
pred_cross_reg = best_model_reg.predict(x_test)

#Calcola MSE sul test set
mse = mean_squared_error(y_test, pred_cross_reg)
print(f"Mean Squared Error on Test Set: {mse}")
#i parametri migliore per il train non sono migliori a predire i valori di test, è un caso di overfitting

Mean Squared Error on Test Set: 6.779187782127585


In [22]:
#Facciamo la cross-validation 5-fold per il SVR
cv_svr = cross_val_score(svr, x_train, y_train, cv=5, scoring='neg_mean_squared_error')

# risultati
print(f"Cross-validation MSE scores: {-cv_svr}")
print(f"Mean cross-validation MSE: {-cv_svr.mean()}")

Cross-validation MSE scores: [7.90676491 6.68448454 6.59563465 6.60482486 5.80959205]
Mean cross-validation MSE: 6.720260205068212


In [23]:
from sklearn.model_selection import RandomizedSearchCV
#Con la Random Search definiamo i parametri che possiamo modificare che per la regressione multipla non sono molti
param_dist = {
    'C': np.logspace(-2, 2, 5),         # Log-scaled for better distribution
    'epsilon': [0.01, 0.1, 0.2, 0.5],   # Epsilon parameter
    'kernel': ['linear', 'poly', 'rbf'], # Kernel type
    'gamma': ['scale', 'auto'],          # Kernel coefficient
}

# random search con la cross-validation
#random_svr = RandomizedSearchCV(estimator=svr, param_distributions=param_dist, n_iter=1, cv=2, scoring='neg_mean_squared_error', random_state=42)
#esecuzione troppo lunga e inefficiente

# Fit della grid search
#random_svr.fit(x_train, y_train)

# Best parameters found by GridSearchCV
#print(f"parametro migliore: {random_svr.best_params_}")
#print(f"Risultato migliore: {-random_svr.best_score_}")

In [24]:
from sklearn.ensemble import VotingRegressor

ensemble = VotingRegressor(
    estimators=[('mul', mul), ('tree', regressor)]
)
ensemble.fit(x_train, y_train)

pred_ensemble = ensemble.predict(x_test)

mse_test = mean_squared_error(y_test, pred_ensemble)

print(f"Mean Squared Error: {mse_test}")
#Il modello con il MSE più basso è l'ensemble tra il la regressione multipla e l'albero

Mean Squared Error: 6.320496019432153


In [25]:
import joblib


#joblib.dump(ensemble, "uova_model.pkl")